In [1]:
from env import host, username, password, get_db_url
import os
import pandas as pd 
import numpy as np

In [2]:
def acquire_zillow_data(use_cache=True):
    '''
    This function returns a snippet of zillow's database as a Pandas DataFrame. 
    When this SQL data is cached and extant in the os directory path, return the data as read into a df. 
    If csv is unavailable, aquisition proceeds regardless,
    reading the queried database elements into a dataframe, creating a cached csv file
    and lastly returning the dataframe for some sweet data science perusal.
    '''

    # If the cached parameter is True, read the csv file on disk in the same folder as this file 
    if os.path.exists('zillow.csv') and use_cache:
        print('Using cached CSV')
        return pd.read_csv('zillow.csv', dtype={'buildingclassdesc': 'str', 'propertyzoningdesc': 'str'})

    # When there's no cached csv, read the following query from Codeup's SQL database.
    print('CSV not detected.')
    print('Acquiring data from SQL database instead.')
    df = pd.read_sql(
        '''
 SELECT
    prop.*,
    predictions_2017.logerror,
    predictions_2017.transactiondate,
    air.airconditioningdesc,
    arch.architecturalstyledesc,
    build.buildingclassdesc,
    heat.heatingorsystemdesc,
    landuse.propertylandusedesc,
    story.storydesc,
    construct.typeconstructiondesc
FROM properties_2017 prop
JOIN (
    SELECT parcelid, MAX(transactiondate) AS max_transactiondate
    FROM predictions_2017
    GROUP BY parcelid
) pred USING(parcelid)
JOIN predictions_2017 ON pred.parcelid = predictions_2017.parcelid
                      AND pred.max_transactiondate = predictions_2017.transactiondate
LEFT JOIN airconditioningtype air USING (airconditioningtypeid)
LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid)
LEFT JOIN buildingclasstype build USING (buildingclasstypeid)
LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid)
LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid)
LEFT JOIN storytype story USING (storytypeid)
LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid)
WHERE prop.latitude IS NOT NULL
  AND prop.longitude IS NOT NULL
  AND transactiondate <= '2017-12-31';             
        '''
                    , get_db_url('zillow'))
    
    df.propertyzoningdesc.astype(str)
    
    
    print('Acquisition Complete. Dataframe available and is now cached for future use.')
    # create a csv of the dataframe for the sake of efficiency. 
    df.to_csv('zillow.csv', index=False)
    
    return df

In [88]:
df = acquire_zillow_data()

Using cached CSV


In [4]:
def describe_data(df):
    '''
    This function takes in a pandas dataframe and prints out the shape, datatypes, number of missing values, 
    columns and their data types, summary statistics of numeric columns in the dataframe, as well as the value counts for categorical variables.
    '''
    # Print out the "shape" of our dataframe - rows and columns
    print(f'This dataframe has {df.shape[0]} rows and {df.shape[1]} columns.')
    print('')
    print('--------------------------------------')
    print('--------------------------------------')
    
    # print the datatypes and column names with non-null counts
    print(df.info())
    print('')
    print('--------------------------------------')
    print('--------------------------------------')
    
    
    # print out summary stats for our dataset
    print('Here are the summary statistics of our dataset')
    print(df.describe().applymap(lambda x: f"{x:0.3f}"))
    print('')
    print('--------------------------------------')
    print('--------------------------------------')

    # print the number of missing values per column and the total
    print('Null Values by Column: ')
    missing_total = df.isnull().sum().sum()
    missing_count = df.isnull().sum() # the count of missing values
    value_count = df.isnull().count() # the count of all values
    missing_percentage = round(missing_count / value_count * 100, 2) # percentage of missing values
    missing_df = pd.DataFrame({'count': missing_count, 'percentage': missing_percentage})\
    .sort_values(by='percentage', ascending=False)
    
    print(missing_df.head(50))
    print(f' \n Total Number of Missing Values: {missing_total} \n')
    df_total = df[df.columns[:]].count().sum()
    proportion_of_nulls = round((missing_total / df_total), 4)
    print(f' Proportion of Nulls in Dataframe: {proportion_of_nulls}\n') 
    print('--------------------------------------')
    print('--------------------------------------')
    
    print('Row-by-Row Nulls')
    print(nulls_by_row(df))
    print('----------------------')


    #print('Relative Frequencies: \n')
    ## Display top 5 values of each variable within reasonable limit
    #limit = 25
    #for col in df.columns:
    #    if df[col].nunique() < limit:
    #        print(f'Column: {col} \n {round(df[col].value_counts(normalize=True).nlargest(5), 3)} \n')
    #    else: 
    #        print(f'Column: {col} \n')
    #        print(f'Range of Values: [{df[col].min()} - {df[col].max()}] \n')
    #    print('------------------------------------------')
    #    print('--------------------------------------')
        
        
def nulls_by_col(df):
    '''
    This function  takes in a dataframe of observations and attributes(or columns) and returns a dataframe where each row is an atttribute name, the first column is the 
    number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute.
    '''
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = (num_missing / rows * 100)
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 
                                 'percent_rows_missing': prcnt_miss})\
    .sort_values(by='percent_rows_missing', ascending=False)
    return cols_missing.applymap(lambda x: f"{x:0.1f}")

def nulls_by_row(df):
    '''
    This function takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, 
    and number of rows with n columns missing.
    '''
    num_missing = df.isnull().sum(axis = 1)
    prcnt_miss = (num_missing / df.shape[1] * 100)
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 
                                 'percent_cols_missing': prcnt_miss})\
    .reset_index()\
    .groupby(['num_cols_missing', 'percent_cols_missing']).count()\
    .rename(index=str, columns={'index': 'num_rows'}).reset_index().set_index('num_cols_missing')\
    .sort_values(by='percent_cols_missing', ascending=False)
    return rows_missing

In [89]:
#describe_data(df)

In [6]:
def remove_columns(df, cols_to_remove):
    '''
    This function takes in a pandas dataframe and a list of columns to remove. It drops those columns from the original df and returns the df.
    '''
    df = df.drop(columns=cols_to_remove)
    return df
                 
                 
def handle_missing_values(df, prop_required_column=0.5 , prop_required_row=0.5):
    '''
    This function takes in a pandas datafeame, default proportion of required columns (set to 50%) and proprtion of required rows (set to 75%).
    It drops any rows or columns that contain null values more than the threshold specified from the original dataframe and returns that dataframe.
    '''
    threshold = int(round(prop_required_column * len(df.index), 0))
    df = df.dropna(axis=1, thresh=threshold)
    threshold = int(round(prop_required_row * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=threshold)
    return df

# combined in one function
def data_prep(df, cols_to_remove=[], prop_required_column=0.5, prop_required_row=0.5):
    '''
    This function calls the remove_columns and handle_missing_values to drop columns that need to be removed. It also drops rows and columns that have more 
    missing values than the specified threshold.
    '''
    df = remove_columns(df, cols_to_remove)
    df = handle_missing_values(df, prop_required_column, prop_required_row)
    return df

In [7]:
# to find the difference 
from collections import Counter

def handle_missing_values(df, prop_required_column=0.5 , prop_required_row=0.5):
    '''
    This function takes in a pandas dataframe, default proportion of required columns (set to 50%) and proprtion of required rows (set to 75%).
    It drops any rows or columns that contain null values more than the threshold specified from the original dataframe and returns that dataframe.
    
    Prior to returning that data, it will print statistics and list counts/names of removed columns/row counts 
    '''
    original_cols = df.columns.to_list()
    original_rows = df.shape[0]
    threshold = int(round(prop_required_column * len(df.index), 0))
    df = df.dropna(axis=1, thresh=threshold)
    threshold = int(round(prop_required_row * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=threshold)
    remaining_cols = df.columns.to_list()
    remaining_rows = df.shape[0]
    dropped_col_count = len(original_cols) - len(remaining_cols)
    dropped_cols = list((Counter(original_cols) - Counter(remaining_cols)).elements())
    print(f'The following {dropped_col_count} columns were dropped because they were missing more than {prop_required_column * 100}% of data: \n{dropped_cols}\n')
    dropped_rows = original_rows - remaining_rows
    print(f'{dropped_rows} rows were dropped because they were missing more than {prop_required_row * 100}% of data')
    return df

In [90]:
#handle_missing_values(df)

In [91]:
#df = data_prep(df)

In [92]:
#describe_data(df)

In [86]:
def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    
    for col in col_list:
        # get quartiles
        q1, q3 = df[f'{col}'].quantile([.25, .75])  
        # calculate interquartile range
        iqr = q3 - q1   
        # get upper bound
        upper_bound = q3 + k * iqr 
        # get lower bound
        lower_bound = q1 - k * iqr   

        # return dataframe without outliers
        
        df = df[(df[f'{col}'] > lower_bound) & (df[f'{col}'] < upper_bound)]
        
    return df

In [12]:
df.parcelid.nunique
#it's showing 77574...but there are only 77319 rows. What's going on? 

<bound method IndexOpsMixin.nunique of 0        14297519
1        17052889
2        14186244
3        12177905
4        10887214
           ...   
77375    10833991
77376    11000655
77377    17239384
77378    12773139
77379    12826780
Name: parcelid, Length: 77380, dtype: int64>

In [13]:
df.propertylandusedesc.unique()

array(['Single Family Residential', 'Condominium',
       'Duplex (2 Units, Any Combination)', 'Residential General',
       'Planned Unit Development', 'Triplex (3 Units, Any Combination)',
       'Quadruplex (4 Units, Any Combination)', 'Cluster Home',
       'Commercial/Office/Residential Mixed Used', 'Cooperative',
       'Manufactured, Modular, Prefabricated Homes', 'Mobile Home',
       'Townhouse'], dtype=object)

In [14]:
df = df[(df.propertylandusedesc == 'Single Family Residential') |
      (df.propertylandusedesc == 'Mobile Home') |
      (df.propertylandusedesc == 'Manufactured, Modular, Prefabricated Homes') |
      (df.propertylandusedesc == 'Cluster Home')]

In [15]:
df.shape

(52784, 34)

In [16]:
df= df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0)]
df.shape

(52595, 34)

In [17]:
#remove properties with 0 baths
df = df[df.bathroomcnt > 0]

In [18]:
df.shape

(52595, 34)

In [19]:
# # keep only properties with square footage greater than 70 (legal size of a bedroom)
df = df[df.calculatedfinishedsquarefeet > 70]

In [20]:
df.shape

(52587, 34)

In [34]:
df2 = df[df.unitcnt <= 1]

In [35]:
df2.shape

(33742, 34)

In [93]:
#df2

In [25]:
(df2.calculatedfinishedsquarefeet == df2.finishedsquarefeet12).sum()
# this indicated thats finishedsquarefeet12 and calculatedfinishedsquarefeet are identical. drop one. 

33742

In [26]:
(df2.fullbathcnt == df2.bathroomcnt).sum()
# completely identical again. Drop it. 

33742

In [27]:
(df2.bathroomcnt == df2.calculatedbathnbr).sum()

33742

In [36]:
df2 = df2.drop(columns= ['finishedsquarefeet12', 'fullbathcnt', 'calculatedbathnbr'])

In [94]:
#describe_data(df2)

https://planning.lacounty.gov/luz/summary/category/residential_zones
## Concerning the Property County land use. 
Mimimum area: 5000 sq feet per lot. 


In [65]:
df3 = df[df.lotsizesquarefeet >= 5000].copy()

In [ ]:
df3

In [66]:
df3.drop(['propertyzoningdesc'], axis=1, inplace=True)
df3 = df3[~df3['propertylandusetypeid'].isin([263, 265, 275])]

In [ ]:
from mitosheet import *; register_analysis('UUID-6c2d09dd-f5b7-4dfe-8001-f7917a2af02e')
    
# Filtered propertycountylandusecode in df3
df3 = df3[df3['propertycountylandusecode'].apply(lambda val: all(val != s for s in ['0103', '0105', '0108', '0109', '010G', '010M', '0111', '0113', '0130', '0131', '0133', '1', '1110', '1111', '1112', '122', '0102']))]


In [70]:
df4 = df3[df3['propertycountylandusecode'] == ('0100', '0101', '0104')]
#.apply(lambda val: all(val == s for s in ['0100', '0101', '0104']))]              

In [76]:
df4 = df3[(df3.propertycountylandusecode == '0100') |
      (df3.propertycountylandusecode == '0101') |
      (df3.propertycountylandusecode == '0104')]

In [ ]:
# now that there are only 27 nans and 13 properties where unit cnt is not equal to one. I can comfortably drop them. 

In [79]:
df5 = df4[df4.unitcnt == 1.0]

In [83]:
# Now I can just drop the nulls from the rest. 
df5 = df5.dropna()

In [104]:
describe_data(df5)

This dataframe has 28459 rows and 33 columns.

--------------------------------------
--------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 28459 entries, 3 to 77379
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            28459 non-null  int64  
 1   parcelid                      28459 non-null  int64  
 2   bathroomcnt                   28459 non-null  float64
 3   bedroomcnt                    28459 non-null  float64
 4   buildingqualitytypeid         28459 non-null  float64
 5   calculatedbathnbr             28459 non-null  float64
 6   calculatedfinishedsquarefeet  28459 non-null  float64
 7   finishedsquarefeet12          28459 non-null  float64
 8   fips                          28459 non-null  float64
 9   fullbathcnt                   28459 non-null  float64
 10  heatingorsystemtypeid         28459 non-null  float64

https://dts.edatatrace.com/dts3/content/doc/whelp/mergedProjects/dts2tt/mergedProjects/dts2ttcs/land_use_la.htm

### concerning property county land use code. 
0100 - Single Residence
0101 Single residence with pool
0104 - Single resident with therapy pool 
( 0102 does not exist, neither does 0103)
anything in 111 counts as commercial, same with 122

In [85]:
print('Relative Frequencies: \n')
# Display top 5 values of each variable within reasonable limit
limit = 25
for col in df5.columns:
    if df5[col].nunique() < limit:
        print(f'Column: {col} \n {round(df5[col].value_counts(normalize=True).nlargest(5), 3)} \n')
    else: 
        print(f'Column: {col} \n')
        print(f'Range of Values: [{df5[col].min()} - {df5[col].max()}] \n')
    print('------------------------------------------')
    print('--------------------------------------')

Relative Frequencies: 

Column: id 

Range of Values: [1439 - 2982055] 

------------------------------------------
--------------------------------------
Column: parcelid 

Range of Values: [10711855 - 13102228] 

------------------------------------------
--------------------------------------
Column: bathroomcnt 
 2.0    0.481
3.0    0.221
1.0    0.215
4.0    0.051
5.0    0.021
Name: bathroomcnt, dtype: float64 

------------------------------------------
--------------------------------------
Column: bedroomcnt 
 3.0    0.484
4.0    0.262
2.0    0.167
5.0    0.066
6.0    0.012
Name: bedroomcnt, dtype: float64 

------------------------------------------
--------------------------------------
Column: buildingqualitytypeid 
 6.0    0.322
8.0    0.262
4.0    0.228
7.0    0.103
9.0    0.033
Name: buildingqualitytypeid, dtype: float64 

------------------------------------------
--------------------------------------
Column: calculatedbathnbr 
 2.0    0.481
3.0    0.221
1.0    0.215
4.0

In [115]:
def prep_zillow(df):
    
    df = data_prep(df)
    
    df = df[(df.propertylandusedesc == 'Single Family Residential') |
      (df.propertylandusedesc == 'Mobile Home') |
      (df.propertylandusedesc == 'Manufactured, Modular, Prefabricated Homes') |
      (df.propertylandusedesc == 'Cluster Home')]
    
    # Remove properties that couldn't even plausibly be a studio. 
    df= df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0)]
    
    # Remove properties where there is not a single bathroom.
    df = df[df.bathroomcnt > 0]
    
 # keep only properties with square footage greater than 70 (legal size of a bedroom)
    df = df[df.calculatedfinishedsquarefeet > 70]
    

    
    # Minimum lot size of single family units.
    df = df[df.lotsizesquarefeet >= 5000].copy()
    
    
    #df = df[~df['propertylandusetypeid'].isin([263, 265, 275])]
    
    # Clear indicators of single unit family. Other codes non-existent or indicate commercial sites. 
   # 0100 - Single Residence
   # 0101 Single residence with pool
   # 0104 - Single resident with therapy pool 
    df = df[(df.propertycountylandusecode == '0100') |
      (df.propertycountylandusecode == '0101') |
      (df.propertycountylandusecode == '0104')]
    
    # With so few remaining nulls it is safe to now control where unit count is only 1 
    df = df[df.unitcnt == 1.0]
    
    # The last nulls can be dropped altogether. 
    df = df.dropna()
    
    # Property where finished area is 152 but bed count is 5. 
    df = df.drop(labels=75325, axis=0)
    
    # This removes the 30 rows where solar is at play. 1% of data. 
    df = df[df['heatingorsystemtypeid'] != 20]
    
    df['yearbuilt'] = df['yearbuilt'].astype(int)
    df.yearbuilt = df.yearbuilt.astype(object) 
    df['age'] = 2017-df['yearbuilt']
    df = df.drop(columns='yearbuilt')
    df['age'] = df['age'].astype('int')
    
    df['central_cooling'] = df.heatingorsystemdesc.map({'Central': 1, 'Floor/Wall': 0})
    
            
    # Redudant columns or uninterpretable columns
    # Unit count was dropped because now its known that theyre all 1. 
    # Finished square feet is equal to calculated sq feet. 
    # full bathcnt and calculatedbathnbr are equal to bathroomcnt
    # property zoning desc is unreadable. 
    # assessment year is unnecessary, all values are 2016. 
    # property land use desc is always single family residence 
    # same with property landuse type id. 
    # room count must be for a different category, as it is always 0.
    # regionidcounty reveals the same information as FIPS. 
    # heatingorsystemtypeid is redundant. Encoded descr. 
    # Id does nothing, and parcelid is easier to represent. 

    
    df.drop(columns= ['finishedsquarefeet12', 'fullbathcnt', 'calculatedbathnbr',
                      'propertyzoningdesc', 'unitcnt', 'propertylandusedesc',
                      'assessmentyear', 'roomcnt', 'regionidcounty', 'propertylandusetypeid',
                      'heatingorsystemtypeid', 'id', 'heatingorsystemdesc' ],
            axis=1, inplace=True)
    
    return df

In [116]:
df2 = prep_zillow(df)

The following 34 columns were dropped because they were missing more than 50.0% of data: 
['airconditioningtypeid', 'architecturalstyletypeid', 'basementsqft', 'buildingclasstypeid', 'decktypeid', 'finishedfloor1squarefeet', 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6', 'fireplacecnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'regionidneighborhood', 'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid', 'yardbuildingsqft17', 'yardbuildingsqft26', 'numberofstories', 'fireplaceflag', 'taxdelinquencyflag', 'taxdelinquencyyear', 'airconditioningdesc', 'architecturalstyledesc', 'buildingclassdesc', 'storydesc', 'typeconstructiondesc']

0 rows were dropped because they were missing more than 50.0% of data


In [118]:
df2.shape

(28377, 22)